# Example 0 - Working with a simple function

In this notebook we will give an overview of the API provided by `malcolm-appraiser` through a simple example.

The math necessary to follow this notebook is explained in [bayesian_inference.md](./bayesian_inference.md).

## Forward model


The forward model for this exemple is given by the function `f(x,y,z)` below:

In [1]:
def f(x, y, z):
    return x*y*z + 2*y*z - 3*x*z + 2

We can evaluate `f` on a few points:

In [2]:
print(f"f(0, 0, 0) = {f(0, 0, 0)}")
print(f"f(3, 0, 3) = {f(3, 0, 3)}")
print(f"f(0, 3, 3) = {f(0, 3, 3)}")

f(0, 0, 0) = 2
f(3, 0, 3) = -25
f(0, 3, 3) = 20


## Boundaries and observation

We will assume that the parameters `x`, `y` and `z` are all comprised between `0` and `3`.

In [3]:
boundaries = [[0,3]]*3

We made some measurements and got
$f(x,y,z) + \nu = 0$
(see [bayesian_inference.md](./bayesian_inference.md) for reference)

where $\nu$ represents the noise of the observation.

We assume this noise can be considered gaussian of mean $0$ and variance $4$.

The goal, for this example, is to evaluate how likely is `x` to be greater than `2`.

Following [bayesian_inference.md](./bayesian_inference.md), we get the likelihood function: 

In [4]:
from math import exp

def likelihood(x, y, z):
    return exp(-(f(x, y, z)**2)/8)

## Sampling and appraisal

### Sampler setup


`malcolm-appraiser` needs to communicate with a server that provides the `malcolm-sampler` gRPC service.

Assuming that the server is up on `localhost` and listening on port `7352`, we provide this information to the appraiser with the following:

In [5]:
import malcolm_appraiser as ma

sampler = ma.MalcolmSampler("localhost:7352")

We then register the boundaries of the problem:

In [6]:
sampler.set_boundaries(boundaries)

### Posterior density

In `malcolm-appraiser`, we approximate the posterior density with a set of points in parameters' space and posterior values.

How to choose which points to measure from is arbitrary and should depend on the task at hand.
Intuitively, we want the set of measurements to contain the most possible information about the actual posterior density.

Here, we will sample uniformely at random and compute posterior values. This will be sufficient for the example.

In [7]:
from random import uniform
from statistics import median

N = 1_000

points = []
for _ in range(N):
    points.append(
        [
            uniform(0, 3),
            uniform(0, 3),
            uniform(0, 3),
        ]
    )

posterior_values = []
for x, y, z in points:
    posterior_values.append(likelihood(x, y, z))

print("maximal posterior value:", round(max(posterior_values), 2))
print("median posterior value: ", round(median(posterior_values), 2))
print("minimal posterior value:", round(min(posterior_values), 2))

maximal posterior value: 1.0
median posterior value:  0.13
minimal posterior value: 0.0


Now we send the information to the sampler:

In [8]:
sampler.set_posterior(points, posterior_values)

### Appraisal

Once the sampler is set, we can generate a set of points according to the posterior density we provided.

In [9]:
samples = sampler.make_samples(15_000)

This set of points is used to compute expectations (i.e. averages).

Here, we want to compute the average of the following `h(x,y,z)`:

In [10]:
from statistics import mean

def h(x, y, z):
    return 1 if x > 2 else 0

probability = mean(map(lambda sample: h(*sample), samples))

print(f"P( x > 2 | f(x,y,z) + nu = 0) ~= {round(probability, 2)}")

P( x > 2 | f(x,y,z) + nu = 0) ~= 0.35
